In [2]:
!pip install mediapipe

# Importing Necessary Packages

In [3]:
import cv2
import mediapipe as mp


In [4]:
mp_drawing=mp.solutions.drawing_utils
mp_hands=mp.solutions.hands
mp_face_detection = mp.solutions.face_detection
mp_holistic = mp.solutions.holistic

- These lines import specific modules from the mediapipe library. mp_drawing is for drawing utilities, and mp_hands is for hand tracking.

### Initilaize Video Capture

In [8]:
# Initialize video capture object
cap = cv2.VideoCapture(0)

# Initialize MediaPipe components
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=4) as hands, \
     mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection, \
     mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        # Read frame from video capture
 
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
 
        # Convert color space from BGR to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
         # Perform face detection
        results_face = face_detection.process(image)
        
        # Draw bounding boxes around detected faces
        if results_face.detections:
            for detection in results_face.detections:
                facebox = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                bbox = int(facebox.xmin * iw), int(facebox.ymin * ih), int(facebox.width * iw), int(facebox.height * ih)
                cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (93, 63, 211), 2)
        
       

        # Perform holistic detection
        results_holistic = holistic.process(image)
        
        """Visualize face landmarks
        if results_holistic.face_landmarks:
            for landmark in results_holistic.face_landmarks.landmark:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 2, (255, 0, 0), -1)"""

         # Perform hand detection
 
        results = hands.process(image)
        
        # Visualize hand landmarks

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4))
                
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
        
        
        # Display the processed image
        cv2.imshow('MediaPipe Hands', image)
        
        # Check for keyboard input to exit
        if cv2.waitKey(1) & 0xFF == 27:
            break

# Release video capture object and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
